lightcurvefinder() is a python implementation of the [Witt (1993)](https://ui.adsabs.harvard.edu/abs/1993ApJ...403..530W/abstract) method to calculate microlensing light curves

Copyright 2020 Robert W. Schmidt rschmidthd at gmail.com

The magnification curve estimation routine iso_magnification() is an extension of causticfinder-py which can be found at https://github.com/rschmidthd/causticfinder-py

__Instructions__: you can run the notebook. It will produce a track image, a caustic pattern (with track overlaid) and a light curve (point source). You can change settings in the first cell.

###### This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

###### This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

###### You should have received a copy of the GNU General Public License along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [ ]:
import matplotlib.pyplot as plt
import cmath as cm
import numpy as np
%matplotlib inline

In [ ]:
# settings
#
#     Infrequently the code seems unable to exit loops. This can be fixed
#     by reducing the step size in the source plane


points_per_curve=200    # critical and caustic curve length = 2*nstars*points_per_curve
points_per_track=20001  # number of points on source track
track_length=2*10       # length of source track in Einstein radii
track_offset=-0.4       # track offset (in x) from the origin
track_angle=0.2         # track angle with respec to y-axis

sigma_c = 0
gamma = 0.0
gamma = gamma / (1-sigma_c)   # reduced shear
normalize = cm.sqrt(abs(1-sigma_c))

nstars = 10
stars_x,stars_y,masses = np.loadtxt('starfile10.txt',unpack=True)

stars=np.zeros(nstars)+np.zeros(nstars)*1j
for i in range(len(stars_x)):
    stars[i]=stars_x[i]*normalize+stars_y[i]*normalize*1j

# define source track
zeta0=-0.5*track_length*np.cos(track_angle)*1j-0.5*track_length*np.sin(track_angle)
zeta_track=np.linspace(zeta0+track_offset,-zeta0+track_offset,points_per_track)

In [ ]:
# lens equation
def beta(z):
    return z + z.conjugate()*gamma + (masses/(stars-z).conjugate()).sum()
# poly_fraction1() and poly_fraction2() are
# auxiliary functions for the Newton-Raphson derivative
def poly_fraction1(z,phi,detj):
    sum1=(-2/(stars-z)).sum()
    sum2=(2*masses/(stars-z)**3).sum()
    sum3=(masses/(stars-z)**2).sum()     
    return sum1 + sum2/(gamma-cm.exp(phi*1j)*cm.sqrt(1-detj)+sum3)  # using Witt eq. 16
def poly_fraction2(z, phi, detj):
    sum1=(masses/(stars-z)**2).sum()
    sum2=(2*masses/(stars-z)**3).sum()    
    return (sum1 + gamma-cm.exp(phi*1j)*cm.sqrt(1-detj))/sum2       # using Witt eq. 16
def detj(z):
    fac=gamma+(masses/(((stars-z).conjugate())**2)).sum()
    return abs(1-fac*fac.conjugate())

def iso_magnification(detj):

    solns=np.zeros(2*nstars)+np.zeros(2*nstars)*1j
    critical_points=np.zeros(2*nstars*points_per_curve)+np.zeros(2*nstars*points_per_curve)*1j
    caustic_points=critical_points.copy()
 
    zp=0.5+0j
    
    # determine initial roots for phi=0
    for m in range(2*nstars):
        
        zp=zp*cm.exp(0.1j)  # allow Newton-Raphson some distance to converge
        norm=abs(zp)
    
        while (norm>1e-6):
            t=zp

            sum=0
            if (m>0):
                sum=(1/(t-solns[:m])).sum()

            zp = t - 1/(poly_fraction1(t,0,detj)-sum)

            norm=abs(t-zp)
        solns[m]=zp
        #print(m,solns)
        
    # follow curves for phi: 0 -> 2 pi
    for n in range(2*nstars):

        zp=solns[n]
        for m in range(points_per_curve):
            t=0
            phi=m*2*cm.pi/points_per_curve
            norm=abs(zp)
    
            while (norm>1e-6):
                t  = zp
                zp = t-poly_fraction2(t,phi,detj)
                norm = abs(t-zp)
            
            #z=beta(zp)
            critical_points[n*points_per_curve+m]=zp
            #caustic_points[n*points_per_curve+m]=z
    
    return critical_points

In [ ]:
# functions relating to the image track

def func(z,zeta):
    return (z+gamma*z.conjugate()+(masses/(stars-z).conjugate()).sum())-zeta
def dfunc(z):
    return (gamma+(masses/((stars-z).conjugate())**2).sum())

def follow_zeta_track(z,p,ct):
    flag=False
    dz_previous=10.0
    dz=1.0
    zp=z*cm.exp(0.1j)

    #print ('start with',ct+p,zeta_track[ct+p])
    
    while (dz<3*dz_previous):                     # criterion to stop at critical curve
        if (p==1) and (ct==len(zeta_track)-1):    # end of line
            flag=True
            break
        if ((p==-1) and ct==1):                   # came back to start
            flag=True
            break

        ct=ct+p                        # negative parity: go backwards
        zeta=zeta_track[ct]
        
        tt=z
        dz_previous=dz
        norm=1.0
        while (norm>1e-6) and (norm < 100):     # stop if convergence (at solution) or divergence (at critical curve)
            t=z
            f=func(t,zeta)
            df=dfunc(t)
            z=t-p*(f-df*f.conjugate())/detj(t)  # note f=zeta(z)-zeta_track[ct], cf Witt eq. 14
            norm=abs(t-z)
        dz=abs(z-tt)
        if (dz<3*dz_previous):                  # criterion top stop at critical curve
            add_image_to_track(z,ct)

    ct=ct-p  # back to before the critical curve
    
    return flag,ct,ximg[-1]+yimg[-1]*1j

def change_isomag_contour(z,p):
    cr = iso_magnification(p*detj(z))
    sep=np.zeros(len(cr))
    zeta=beta(z)
    for x in range(len(cr)):
        sep[x]=abs(beta(cr[x])-zeta)
    zp=cr[np.argsort(sep)][0]
    return zp

def find_initial_primary_position(z, zeta):    # Witt eq. 11
    norm=1.0
    while (norm>1e-6):
        t=z
        z = zeta - gamma*t.conjugate()-(masses/(stars-t).conjugate()).sum()
        norm=abs(t-z)
    return z

def find_initial_secondary_position(zeta):    # Witt eq. 13
    sols=np.zeros(nstars)+np.zeros(nstars)*1j
    for x in range(nstars):
        sum=gamma*stars[x]
        for y in range(nstars):
            if y!=x:
                sum=sum+(masses[y]/(stars[y]-stars[x]))
        sols[x]=stars[x]-masses[x]/(zeta.conjugate()-stars[x].conjugate()-sum)
    return sols

def add_image_to_track(z,ct):
    ximg.append(z.real)
    yimg.append(z.imag)
    mag[ct]=mag[ct]+1/detj(z)

In [ ]:
# lightcurvefinder main
#
# - The code first follows the primary track, which ends at either the track end or at a star.
#     If the primary track encounters a critical curve, the code will cross the critical curve,
#     and follow the track further (can happen several times).
# - Then the code follows all secondary image tracks, which all start near a star.
#     Secondary tracks are followed backwards to make sure that the primary track
#     also gets completed if it happened to end at a star.
# - Critical curves are determined if the image positions for consecutive source positions diverge
#     (currently by a factor 3)
#
# this procedure is a modified version of the one described in Witt 1993, ApJ 403, 530

def lightcurvefinder():
    #initial position primary track
    zeta=zeta_track[0]
    zp=find_initial_primary_position(zeta*cm.exp(0.01j), zeta)

    ct=0                           # ct is the source track index
    add_image_to_track(zp,ct)

    p=1
    flag,ct,z=follow_zeta_track(zp,p,ct)

    # if flag is False, the image track has hit a critical line
    while (not flag):
        # change isomag-line
        p=p*-1
        zp=change_isomag_contour(z,p)
        #add_image_to_track(zp,ct)

        # follow track
        flag,ct,z=follow_zeta_track(zp,p,ct)
    print ('primary done')

    # initial positions secondary tracks
    zeta=zeta_track[-1]
    secondary_initial=find_initial_secondary_position(zeta)

    for zp in secondary_initial:
        print ('doing secondary')
        p=-1                       # negative parity images near the star positions
        ct=len(zeta_track)-1
        flag,ct,z=follow_zeta_track(zp,p,ct)
        while (not flag):
            p=p*-1
            zp=change_isomag_contour(z,p)
            #add_image_to_track(zp,ct)
            
            flag,ct,z=follow_zeta_track(zp,p,ct)

    print ('done')
    return flag   

In [ ]:
# running lightcurvefinder()

mag=np.zeros(points_per_track)
ximg=[]
yimg=[]
state = lightcurvefinder()

In [ ]:
# plotting the image track (blue), the stars (orange) and critical curves (green)

plt.plot(ximg,yimg,'.',markersize=0.9,color='blue')
critical = iso_magnification(0.0) 
for i in range(2*len(stars)):
    plt.plot(np.real(critical[i*points_per_curve:(i+1)*points_per_curve]),
             np.imag(critical[i*points_per_curve:(i+1)*points_per_curve]),color='green')

plt.scatter(stars_x,stars_y,color='orange')
plt.ylim(-6,6)
plt.xlim(-6,6)

In [ ]:
# plotting the caustics and the track

critical = iso_magnification(0.0) 
caustic = []
for z in critical:
    caustic.append(beta(z))

plt.axes().set_aspect('equal')

for i in range(2*len(stars)):
    plt.plot(np.real(caustic[i*points_per_curve:(i+1)*points_per_curve]),
             np.imag(caustic[i*points_per_curve:(i+1)*points_per_curve]),color='green')

plt.plot(zeta_track.real,zeta_track.imag,color='blue')

plt.xlim(-6,6)
plt.ylim(-6,6)

In [ ]:
# Note the mean magnification normally needs to be defined for the simulation
# using kappa and gamma
#
mean_mag = mag.mean()
plt.plot(-2.5*np.log(mag[4000:16000]/mean_mag)/np.log(10))
plt.ylim(1,-3.5)